In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from collections import defaultdict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import pickle
import os
from pathlib import Path
from datetime import datetime
import polars as pl
# pl deals with parquet files efficiently
import gc
from glob import glob
# glob search for files more effectively
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ROOT = Path("/content/drive/MyDrive/dataset/")
TRAIN_DIR_ALL = ROOT / "parquet_files" / "train"
TRAIN_DIR = ROOT / "parquet_files" / "selected_train"
TEST_DIR  = ROOT / "parquet_files" / "test"

#### Download kaggle dataset for the first time

In [ ]:
# Installs the Kaggle API Client
! pip install -q kaggle

# Upload kaggle.json file into Files
# from google.colab import files
# files.upload()

# Make directory named kaggle and copy kaggle.json file there
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

# Change the permissions of the file
! chmod 600 ~/.kaggle/kaggle.json

# You can check if everything's okay by running this command
# This display all datasets available on Kaggle via the Kaggle API
! kaggle datasets list

# Download competition datasets from Kaggle via the Kaggle API
! kaggle competitions download -c home-credit-credit-risk-model-stability
! mkdir ~/.kaggle/dataset
! unzip home-credit-credit-risk-model-stability -d ~/.kaggle/dataset

In [ ]:
# Copy the dataset into MyDrive in Google Drive
!cp -r /root/.kaggle/kaggle.json /content/drive/MyDrive
!cp -r /root/.kaggle/dataset /content/drive/MyDrive

#### Data is too big, we randomly select some data

In [ ]:
import random

# Define the range, subset size, and random seed
total_range = 3000000
subset_size = 100000

# Set the random seed
random.seed(42)

# Generate a random subset of the specified size
subset = random.sample(range(1, total_range + 1), subset_size)

# Get all test ids
test_ids = set()
for filename in os.listdir(ROOT / "parquet_files" / "test"):
    df = pl.read_parquet(ROOT / "parquet_files" / "test" / filename)
    test_ids |= set(df['case_id'])
test_ids

# Union to form the selected ids
selected_ids = set(subset) | test_ids

In [ ]:
os.listdir(TRAIN_DIR_ALL)

In [ ]:
filename = "train_base.parquet"
df = pl.read_parquet(TRAIN_DIR_ALL / filename)
df = df.filter(pl.col('case_id').is_in(selected_ids))

In [ ]:
df.write_parquet(TRAIN_DIR / filename)
pl.read_parquet(TRAIN_DIR / filename)

In [ ]:
# multiple storage for "train_credit_bureau_a_2_5.parquet" since it is an unusually large file

'''
col_name = pl.read_csv(ROOT / "csv_files/train" / "train_credit_bureau_a_2_5.csv", n_rows=10000000).columns
df = pl.read_csv(ROOT / "csv_files/train" / "train_credit_bureau_a_2_5.csv", skip_rows=30000000, n_rows=10000000)
df.columns = col_name
df = df.filter(pl.col('case_id').is_in(selected_ids))
df.write_parquet(ROOT / "parquet_files/selected_train" / "train_credit_bureau_a_2_5_3.parquet")
pl.read_parquet(ROOT / "parquet_files/selected_train" / "train_credit_bureau_a_2_5_3.parquet")
'''